<a href="https://colab.research.google.com/github/yimstar9/Engineer-Big-Data-Analysis_R/blob/main/2%ED%9A%8C_%EC%9E%91%EC%97%852%ED%98%95_%EA%B8%B0%EC%B6%9C_2%ED%9A%8C%EB%8F%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.kaggle.com/code/limmyoungjin/tutorial-t2-2-r/notebook

# %% [markdown]
# ## 전자상거래 배송 데이터
# ### 제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기
# 학습용 데이터 (X_train, y_train)을 이용하여 배송 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)
# 
# ![image.png](attachment:f70c3a4b-9984-4656-af95-dac047a900cb.png)
# 
# [시험용 데이터셋 만들기] 코드는 예시문제와 동일한 형태의 X_train, y_train, X_test 데이터를 만들기 위함임
# 
# (유의사항)
# - 성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, 피처엔지니어링, 분류알고리즘, 하이퍼파라미터 튜닝, 모형 앙상블 등이 수반되어야 한다.
# - 수험번호.csv파일이 만들어지도록 코드를 제출한다.
# - 제출한 모델의 성능은 ROC-AUC형태로 읽어드린다.

# %% [code] {"jupyter":{"outputs_hidden":false},"execution":{"iopub.status.busy":"2022-11-23T14:53:03.255441Z","iopub.execute_input":"2022-11-23T14:53:03.257002Z","iopub.status.idle":"2022-11-23T14:53:07.143993Z"}}
library(dplyr)
library(randomForest)
library(caret)
library(ModelMetrics)
df<-read.csv('../input/customer-analytics/Train.csv')


#결측치 없음
#colSums(is.na(df))


df<-df%>%mutate_at(vars(Warehouse_block,Mode_of_Shipment,Product_importance,Gender,Reached.on.Time_Y.N),as.factor)
p<-preProcess(df,"range")
df$Weight_in_gms<-predict(p,df)$Weight_in_gms

set.seed(99)
idx<-sample(1:nrow(df),0.7*nrow(df))
train<-df[idx,]
test<-df[-idx,]
str(train)
m_rf<-randomForest(Reached.on.Time_Y.N~.,train[,-1],ntree=300)
p_rf<-predict(m_rf,test[,-c(1,12)])

caret::confusionMatrix(test$'Reached.on.Time_Y.N',p_rf)$byClass
caret::confusionMatrix(test$'Reached.on.Time_Y.N',p_rf)$overall
auc(test$'Reached.on.Time_Y.N',p_rf)

result<-cbind(test[,1],as.character(p_rf))
colnames(result)<-c("ID","Reached.on.Time_Y.N")
write.csv(result,"009.csv",row.names=F)
zz<-read.csv("009.csv")
head(zz)
